# • End to End Walkthrough

> Hierarchical Forecast's reconciliation and evaluation.

This notebook offers a step to step guide to create a hierarchical forecasting pipeline.

In the pipeline we will use `HierarchicalForecast` and `StatsForecast` core class, to create base predictions, reconcile and evaluate them. 

We will use the TourismL dataset that summarizes large Australian national visitor survey.

Outline
1. Installing Packages
2. Prepare TourismL dataset
    - Read and aggregate
    - StatsForecast's Base Predictions
3. Reconciliar
4. Evaluar

<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/TourismLarge-Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Installing HierarchicalForecast

In [ ]:
# %%capture
# !pip install hierarchicalforecast
# !pip install -U numba statsforecast datasetsforecast

In [ ]:
import os
import numpy as np
import pandas as pd

from datasetsforecast.hierarchical import HierarchicalData

from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive

from hierarchicalforecast.utils import HierarchicalPlot
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, MinTrace, ERM

## 2. Preparing TourismL Dataset

### 2.1 Read and Aggregate

In [ ]:
Y_df, S_df, tags = HierarchicalData.load('./data', 'TourismLarge')
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

In [ ]:
hplot = HierarchicalPlot(S=S_df, tags=tags)
hplot.plot_summing_matrix()

In [ ]:
Y_test_df = Y_df.groupby('unique_id').tail(12)
Y_train_df = Y_df.drop(Y_test_df.index)

Y_test_df = Y_test_df.set_index('unique_id')
Y_train_df = Y_train_df.set_index('unique_id')

In [ ]:
Y_train_df

### 2.2 StatsForecast's Base Predictions

In [ ]:
%%capture
if os.path.isfile('Y_hat.csv'):
    Y_hat_df = pd.read_csv('Y_hat.csv')
    Y_hat_df = Y_hat_df.set_index('unique_id')
else:
    fcst = StatsForecast(
        df=Y_train_df, 
        models=[AutoARIMA(season_length=12)],
        fallback_model=[Naive()],
        freq='M', 
        n_jobs=-1
    )
    Y_hat_df = fcst.forecast(h=12, level=[80])
    Y_hat_df.to_csv('Y_hat.csv', index=False)

In [ ]:
Y_hat_df

## 3. Reconciliate Predictions

In [ ]:
# reconcilers = [
#     BottomUp(),
#     MinTrace(method='ols'),
#     ERM(method='reg_bu', lambda_reg=100)
# ]
# hrec = HierarchicalReconciliation(reconcilers=reconcilers)
# Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_train_df, 
#                           S=S_df, tags=tags)

## 4. Evaluation

For reference we can check the performance when compared to that reported by previous hierarchical forecasting studies from the [PMM-CNN paper](https://arxiv.org/abs/2110.13179).

## References